In [1]:
import os, nltk
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
os.getcwd()

'c:\\Users\\T-Gamer\\Documents\\SideDrive\\UFMA\\2022.1\\Topicos Especiais (NLP)\\Exercicios\\Trabalho Final\\Implementação\\source'

In [2]:
def get_word2vec_embeddings(dataset:pd.DataFrame, verbose:bool=False, dim:int=200) :
    sentences = []
    for sentence in list(dataset["text"]) :
        sentences.append(sentence.split())
    if verbose : 
        print("No. of Sentences:", len(sentences))
        for sentence in sentences[ : 5] : print("   ", sentence)
    
    model = Word2Vec(sentences=sentences, vector_size=dim, window=5, min_count=1, workers=4)
    model.train(sentences, epochs=100, total_examples=model.corpus_count, total_words=model.corpus_total_words, compute_loss=True)
    
    words = model.wv.index_to_key
    if verbose : 
        print("No. of Words:", len(words))
        for word in words[ : 5] : print("   ", word)
    
    vectors = []
    for word in words :
        vectors.append(model.wv[word])
    vectors = np.array(vectors)
    
    embeddings = pd.DataFrame(vectors, index=words, columns=range(1, dim + 1))
    
    return embeddings

In [8]:
# Para datasets com k_folding
# Args
k = 10
dataset_folder = "../resources/datasets/TwitterAirlines"
dataset_name = "TweetsProcessed2"
# dataset_folder = "../resources/datasets/IMDBMovieReviews"
# dataset_name = "IMDB_MRProcessed"

# Embeddings
save_path = f"{dataset_folder}/embeddings"
if not os.path.exists(save_path) : os.makedirs(save_path)

folds = [pd.read_csv(f"{dataset_folder}/folds/{dataset_name}_Fold{i_fold + 1}.csv", index_col=0) for i_fold in range(k)]

fold_embeddings = []
for i_fold in range(k) :
    trainset = pd.concat(folds[ : i_fold] + folds[i_fold + 1 : ])
    embeddings = get_word2vec_embeddings(trainset, dim=300)
    embeddings.to_csv(f"{save_path}/{dataset_name}_Fold{i_fold + 1}.csv")
    fold_embeddings.append(embeddings)
fold_embeddings[0]

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
to,0.516435,0.347733,-0.452792,0.088457,0.039299,-2.245621,1.647378,1.310579,0.678390,-0.790236,...,0.411603,0.290859,0.107042,-0.779314,0.259411,-0.152420,0.708290,0.060583,-0.517372,-1.263536
i,-0.219392,0.868007,-0.940412,-1.694158,1.072378,-1.477332,0.058441,0.849889,1.005164,-2.197675,...,1.270338,-0.148660,-1.975768,-0.748865,0.396474,0.917793,1.186704,-0.799262,0.320459,0.495669
the,0.812115,0.603371,-0.977580,-0.547784,-1.165419,-0.078007,0.598055,-0.043790,-0.164646,-1.004631,...,0.325372,0.545254,-0.082030,-0.105519,0.529428,-0.888825,0.611942,0.176268,0.943490,-0.552509
a,-0.253051,0.079646,-1.244305,-0.151374,-0.657647,-1.130011,0.555372,0.182149,1.623513,-1.580246,...,1.141599,-0.267230,-0.192988,0.521902,-0.074032,-0.591412,0.012265,0.805719,0.290775,-0.737753
you,-0.101054,-1.602474,0.512774,-0.871327,1.375609,-0.115770,-0.134071,0.939963,0.994703,0.127985,...,1.199927,0.329443,-1.701032,-0.476374,-0.388293,-0.936725,0.953041,0.859887,0.123127,0.069152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,-0.117521,0.105164,-0.230925,0.031505,-0.046003,-0.292806,0.014699,0.349486,0.215918,-0.007597,...,0.256132,-0.010253,0.069876,-0.095160,0.115071,0.206615,0.151744,-0.167131,0.171076,-0.194836
arrangement,0.132880,0.052602,0.177719,0.024279,0.189649,-0.400314,-0.137722,0.111640,-0.050551,0.070838,...,0.156535,0.319833,0.181603,0.028461,0.150362,0.120446,0.077672,-0.094794,-0.127063,-0.133468
cncled,0.146069,0.248842,-0.122057,-0.151805,0.027676,0.069226,-0.375591,0.218387,0.116919,0.159562,...,0.132486,0.417452,-0.026694,-0.003067,0.029017,0.125136,-0.216179,-0.022970,0.205853,0.280056
0223,0.207825,0.396298,-0.270329,-0.145979,0.077503,0.020073,-0.250553,0.187721,0.208813,0.066279,...,0.085990,0.277539,0.062296,0.027458,-0.034936,0.013833,-0.206100,-0.005330,0.326830,0.210818


In [3]:
# Para dataset simples
# Args
resource_folder_name = "../resources"
dataset_folder_name = "StanfordSentimentTreebank"
dataset_name = "SST2Processed2-train"
dim = 768

# Exec
trainset = pd.read_csv(f"{resource_folder_name}/datasets/{dataset_folder_name}/split/{dataset_name}.csv")
embeddings = get_word2vec_embeddings(trainset, dim=dim)
embeddings.to_csv(f"{resource_folder_name}/embeddings/{dataset_folder_name}/{dataset_name}_dim{dim}.csv")
embeddings

,1,2,3,4,5,6,7,8,9,10,...,759,760,761,762,763,764,765,766,767,768
<start>,-0.034255,0.169965,-0.363289,0.506442,-0.427822,-0.150762,0.148484,0.252551,-0.354949,-0.372175,...,-0.298481,0.413810,0.289104,-0.229494,0.547424,-0.645688,-0.039345,0.678340,-0.932603,-0.236194
the,0.217823,-0.161214,0.394072,0.068987,0.046122,0.154046,-0.454703,0.359115,0.021212,-0.538390,...,-0.001280,0.295897,0.398796,-1.331156,0.053657,-0.017822,0.128449,0.474252,-0.645320,1.193220
a,0.167699,-0.140574,-0.182011,0.753963,0.042203,0.163304,0.138383,0.016099,-0.236057,-0.259253,...,-0.174040,0.231640,0.223893,-0.948751,0.399870,-0.272608,-0.146616,0.261264,0.009641,0.473784
and,-0.104824,0.308404,-0.035850,0.195959,0.277350,1.062822,0.031705,0.053349,0.142076,-0.109975,...,0.008958,-0.329107,0.387872,-0.461119,0.581441,0.248812,-0.531152,-0.077007,-0.493601,0.707036
of,-0.189153,-0.038014,0.178236,-0.094203,0.081555,0.331777,-0.471609,-0.064213,-0.952977,0.116187,...,-0.088022,0.850869,0.420592,-0.957811,0.035724,-1.089710,-0.550777,1.008658,0.052567,0.297134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
weirder,0.013368,0.089305,0.110290,0.035707,-0.029785,0.110641,-0.129766,0.096415,-0.014302,-0.029318,...,-0.063590,-0.049419,0.003443,0.006120,0.051539,-0.043003,0.129099,-0.067617,-0.005751,0.006141
nurtures,-0.047219,0.003882,-0.042473,0.050664,0.008885,0.082935,-0.066954,-0.038774,0.083825,0.112258,...,0.077044,0.007705,-0.012869,0.062868,0.021358,0.081706,-0.111334,0.050834,0.060996,-0.072740
duckling,-0.010130,0.024509,0.053648,0.065153,0.075865,0.007628,0.061000,-0.092785,-0.055394,-0.200623,...,-0.058598,0.096407,-0.009398,-0.195503,-0.211286,0.058663,0.139345,-0.016107,0.003114,0.144125
regrets,0.011201,0.055083,0.041088,0.044559,0.040724,-0.094580,0.030258,-0.044762,0.002380,-0.000654,...,0.014300,-0.098278,-0.083174,0.045312,0.024754,-0.066009,-0.005088,-0.020318,-0.017686,-0.028166
